### Output on one channel, read input

In [1]:
import nidaqmx.system as ni

system = ni.System.local()
for device in system.devices:
    print(device)

# device = system.devices['Dev1']
# device == nidaqmx.system.Device('Dev1')
    
# phys_chan = device.ai_physical_chans['ai0']

nidaqmx.constants.AOIdleOutputBehavior = 12526

with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    task.write([4], auto_start=True)

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai7")
    print(task.read())
    
# with nidaqmx.Task() as task:
#     task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
#     rd = task.read() 

# print(rd)

ModuleNotFoundError: No module named 'nidaqmx'

### Put output on one channel and read several channels, print everything

In [ ]:
import pprint
import nidaqmx

pp = pprint.PrettyPrinter(indent=4)

with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    task.write([4], auto_start=True)

with nidaqmx.Task() as task:
#     task.ai_channels.add_ai_voltage_chan("Dev1/ai0")

#     print('1 Channel 1 Sample Read Raw: ')
#     data = task.read()
#     pp.pprint(data)

#     print('1 Channel N Samples Read Raw: ')
#     data = task.read(number_of_samples_per_channel=8)
#     pp.pprint(data)

    task.ai_channels.add_ai_voltage_chan("Dev1/ai5:7")

    print('N Channel 1 Sample Read: ')
    data = task.read()
    pp.pprint(data)

    print('N Channel N Samples Read: ')
    data = task.read(number_of_samples_per_channel=8)
pp.pprint(data)

###  An example of how to use PyDAQmx to acquire 1000 points

In [ ]:
from PyDAQmx import *
import numpy

analog_input = Task()
read = int32()
data = numpy.zeros((1000,), dtype=numpy.float64)

# DAQmx Configure Code
analog_input.CreateAIVoltageChan(b"Dev1/ai0",b"",DAQmx_Val_Cfg_Default,
                                 -10.0,10.0,DAQmx_Val_Volts,None)
analog_input.CfgSampClkTiming(b"",10000.0,DAQmx_Val_Rising,DAQmx_Val_FiniteSamps,1000)

# DAQmx Start Code
analog_input.StartTask()

# DAQmx Read Code
analog_input.ReadAnalogF64(1000,10.0,DAQmx_Val_GroupByChannel,data,1000,byref(read),None)

print("Acquired %d points"%read.value)
print(len(data))

## This example is a PyDAQmx version of the ContAcq_IntClk.c example

It illustrates the use of callback functions

This example demonstrates how to acquire a continuous amount of
data using the DAQ device's internal clock. It incrementally stores the data
in a Python list.

In [ ]:
from PyDAQmx import *
import numpy
from ctypes import *

from PyDAQmx import Task
from numpy import zeros

class CallbackTask(Task):
    def __init__(self):
        Task.__init__(self)
        self.data = zeros(1000)
        self.a = []
        self.CreateAIVoltageChan(b"Dev1/ai0",b"",DAQmx_Val_RSE,
                                 -10.0,10.0,DAQmx_Val_Volts,None)
        self.CfgSampClkTiming(b"",10000.0,DAQmx_Val_Rising,DAQmx_Val_ContSamps,1000)
        self.AutoRegisterEveryNSamplesEvent(DAQmx_Val_Acquired_Into_Buffer,1000,0)
        self.AutoRegisterDoneEvent(0)
    def EveryNCallback(self):
        read = int32()
        self.ReadAnalogF64(1000,10.0,DAQmx_Val_GroupByScanNumber,self.data,
                           1000,byref(read),None)
        self.a.extend(self.data.tolist())
        print(self.data[0])
        return 0 # The function should return an integer
    def DoneCallback(self, status):
        print("Status",status.value)
        return 0 # The function should return an integer


task=CallbackTask()
task.StartTask()

input('Acquiring samples continuously. Press Enter to interrupt\n')

task.StopTask()
task.ClearTask()

In [ ]:
print(len(task.a))

N Channel 1 Sample Read: 
[2.454756264109159, 1.5471475915227697, 4.000121790712001]
N Channel N Samples Read: 
[   [   2.5034603692745483,
        2.7083137229024854,
        2.884536891266446,
        3.0364079688721795,
        3.167382335168061,
        3.2802571990549128,
        3.377829763755347,
        3.461909984904626],
    [   1.6759833989781971,
        1.818805161441118,
        1.967056726581752,
        2.1161309400815878,
        2.2629015235338414,
        2.405393989524154,
        2.5416338531355636,
        2.671127498964194],
    [   3.999957250348825,
        3.999957250348825,
        3.999628169622322,
        3.999792709985599,
        3.999957250348825,
        4.000121790712001,
        3.999628169622322,
        3.999792709985599]]
